In [25]:
from pyincore import IncoreClient
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil

In [26]:
# Connect to IN-CORE service
client = IncoreClient()

Connection successful to IN-CORE services. pyIncore version detected: 0.9.0


## Joplin, MO tornado

In [27]:
# The simulated EF-5 tornado shows geographical locations and the range
# of wind speed of tornado hazard in Joplin.
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"

### Building damage

In [28]:
bldg_dmg = BuildingDamage(client)

In [29]:
bldg_dataset_id = "5dbc8478b9219c06dd242c0d"  # Load the Joplin building dataset, which is a package of GIS files.
mapping_id = "5d8942dfb9219c068ea795ea"  # Create a mapping to assign tornado fragilities to 19 building archetypes.
fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


In [30]:
result_name = "Joplin_bldg_dmg_result"
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)  # number of CPU to run the model parallelly

True

In [31]:
bldg_dmg.run_analysis()

True

In [32]:
bldg_dmg_result = bldg_dmg.get_output_dataset('ds_result')

### Population dislocation

In [33]:
# Joplin, MO, House unit allocation, 2ev2_housingunitallocation_1238
housing_unit_alloc = "5dc1c196b9219c06dd2e3f0b"
# Joplin, MO, Background data, 2ev2_2019-08-01_bgdata
bg_data = "5d4c9545b9219c0689b2358a"

# Value loss parameters, value_loss_bai09.csv
value_loss = "60354810e379f22e16560dbd"

In [34]:
# Create population dislocatin 
pop_dis = PopulationDislocation(client)

# Load input dataset
# Joplin, MO, Building damage result
pop_dis.set_input_dataset("building_dmg", bldg_dmg_result)
pop_dis.load_remote_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_poss_param", value_loss)

# Specify the result name
result_name = "IN-CORE_1bv6_population_dislocation"

seed = 1111

# Set analysis parameters
pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [35]:
# Run Population dislocation analysis
pop_dis.run_analysis()

True

In [36]:
# Retrieve result dataset
result = pop_dis.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
df.head()

,guid,DS_0,DS_1,DS_2,DS_3,strctid,statefp10,tractce10,pumgeoid10,pumname10,...,bgyear,pblackbg,phispbg,d_sf,rploss_0,rploss_1,rploss_2,rploss_3,prdis,dislocated
0,59fc3532-2c7d-4d68-b7d5-5a7b6ed4a556,1.0,0.0,0.0,0.0,S0021177,29.0,10100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,7.269338,3.075489,0,0.002914,0.100425,0.524702,0.889881,0.0,False
1,535a461c-f028-4e05-9100-43541b645f1c,1.0,0.0,0.0,0.0,S0023095,29.0,10100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,7.269338,3.075489,0,0.006930,0.132025,0.677548,0.951019,0.0,False
2,38688790-543b-4a28-9530-49975d353de4,1.0,0.0,0.0,0.0,S0021178,29.0,10100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,7.269338,3.075489,0,0.005776,0.100000,0.654935,0.941974,0.0,False
3,e791aea5-706d-4e59-b4e9-0eada663b6c7,1.0,0.0,0.0,0.0,S0023093,29.0,10100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,7.269338,3.075489,0,0.002404,0.105851,0.436479,0.854591,0.0,False
4,46b98fd2-f0d0-4ef4-9de3-ea1798e93050,1.0,0.0,0.0,0.0,S0023044,29.0,10100.0,2902800.0,Jasper & Newton Counties PUMA,...,2010.0,7.269338,3.075489,0,0.003029,0.133077,0.438777,0.855511,0.0,False
